In [1]:
import pandas as pd
import numpy as np
import xgboost
#import matlpotlib.pyplot as plt
import pickle

In [2]:
#pip install --upgrade numpy

In [3]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [4]:
#pip install xgboost

In [5]:
data_path = 'data/train_df.csv'
df = pd.read_csv(data_path)
df.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,VRDeck,Name,Transported,GroupId,GrpPassId,Deck,Decknum,DeckSide,Expenditure,Adult
0,0001_01,Europa,True,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,0.0,Maham Ofracculy,False,1,1,B,0.0,P,0.0,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,44.0,Juanna Vines,True,2,1,F,0.0,S,736.0,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,49.0,Altark Susent,False,3,1,A,0.0,S,10383.0,1


In [6]:
list(df.columns)

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported',
 'GroupId',
 'GrpPassId',
 'Deck',
 'Decknum',
 'DeckSide',
 'Expenditure',
 'Adult']

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
 14  GroupId       8693 non-null   int64  
 15  GrpPassId     8693 non-null   int64  
 16  Deck          8494 non-null   object 
 17  Decknum       8494 non-null   float64
 18  DeckSide      8494 non-null 

In [8]:
"""list_cat = [[True,False],['Earth','Europa','Mars'],['TRAPPIST-1e','Cancri e','PSO J318.5-22'],['F','E','D','C','B','A','T'],['S','P']]
o_enc = OrdinalEncoder(categories=cat_cols)
df = o_enc.fit_transform(df)
df.head(3)"""

"list_cat = [[True,False],['Earth','Europa','Mars'],['TRAPPIST-1e','Cancri e','PSO J318.5-22'],['F','E','D','C','B','A','T'],['S','P']]\no_enc = OrdinalEncoder(categories=cat_cols)\ndf = o_enc.fit_transform(df)\ndf.head(3)"

In [9]:
df.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,VRDeck,Name,Transported,GroupId,GrpPassId,Deck,Decknum,DeckSide,Expenditure,Adult
0,0001_01,Europa,True,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,0.0,Maham Ofracculy,False,1,1,B,0.0,P,0.0,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,44.0,Juanna Vines,True,2,1,F,0.0,S,736.0,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,49.0,Altark Susent,False,3,1,A,0.0,S,10383.0,1


In [10]:
df.drop(labels=['Name','Cabin'],axis=1,inplace=True)

In [11]:
cat_cols = list(df.select_dtypes('object').columns.values)
cat_cols.remove('PassengerId')
cat_cols

['HomePlanet', 'Destination', 'VIP', 'Deck', 'DeckSide']

In [12]:
#df['HomePlanet'].value_counts()
for col in cat_cols:
    print(col)
    print(df[col].value_counts())

HomePlanet
Earth     4772
Europa    2162
Mars      1759
Name: HomePlanet, dtype: int64
Destination
TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64
VIP
False    8291
True      199
Name: VIP, dtype: int64
Deck
F    2794
G    2559
E     876
B     779
C     747
D     478
A     256
T       5
Name: Deck, dtype: int64
DeckSide
S    4288
P    4206
Name: DeckSide, dtype: int64


In [13]:
tf = {True:1,False:0}
home_planet = {'Earth':3,'Europa':3,'Mars':1}
dest_planet = {'TRAPPIST-1e':3,'Cancri e':2,'PSO J318.5-22':1}
deck = {'F':7,'E':6,'D':5,'C':4,'B':3,'A':2,'T':1}
port = {'S':0,'P':1}


In [14]:
df['CryoSleep'] = df['CryoSleep'].map(tf)
df['Destination'] = df['Destination'].map(dest_planet)
df['HomePlanet'] = df['HomePlanet'].map(home_planet)
df['VIP'] = df['VIP'].map(tf)
df['Deck'] = df['Deck'].map(deck)
df['DeckSide'] = df['DeckSide'].map(port)
df['Transported'] = df['Transported'].map(tf)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   int64  
 2   CryoSleep     8693 non-null   int64  
 3   Destination   6711 non-null   float64
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   float64
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Transported   8693 non-null   int64  
 12  GroupId       8693 non-null   int64  
 13  GrpPassId     8693 non-null   int64  
 14  Deck          5935 non-null   float64
 15  Decknum       8494 non-null   float64
 16  DeckSide      8494 non-null   float64
 17  Expenditure   7785 non-null   float64
 18  Adult         8693 non-null 

In [16]:
X = df.drop(labels=['Transported','PassengerId'],axis=1)
y = df['Transported']

In [17]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [44]:
def train_xgb_model(parameters):
    model = xgboost.XGBClassifier(num_boost_round=40)
    model.set_params(**parameters)
#    model = xgboost.XGBClassifier(n_estimators = parameters['n_estimator'],objective = parameters['objective'], colsample_bytree = parameters['colsample_bytree'], learning_rate = parameters['learning_rate'],
#                max_depth = parameters['max_depth'], alpha = parameters['alpha'])
    return model
def fit_model(model,X_train,y_train):
    model.fit(X_train,y_train)
    
def predict_results(model,X):
    predicted_values = model.predict(X)
    return predicted_values
    
def calculate_acc(model,y_actual,y_predicted):
    rmse = accuracy_score(y_actual,y_predicted)
    return rmse
    
def retune_model(model,hyperparameter_grid,X_train,y_train):
        
        grid_search_cv = RandomizedSearchCV(
        estimator=model,
        param_distributions=hyperparameter_grid,
        cv=10,n_iter=100,
        scoring = 'neg_mean_absolute_error',
        n_jobs=4,
        verbose=5,
        return_train_score=True,
        random_state=42
        )
        grid_search_cv.fit(X_train,y_train)
        return grid_search_cv
    

In [19]:
parameters = {'objective' :'reg:squarederror', 'colsample_bytree' : 0.3, 'learning_rate' : 0.1,
                'max_depth' : 5, 'alpha' : 10, 'n_estimator' : 10}
model_1 = train_xgb_model(parameters)
fit_model(model_1,x_train,y_train)
model_1.fit(x_train,y_train)
y_pred = predict_results(model_1,x_test)
acc = calculate_acc(model_1,y_test,y_pred)


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:39:25] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:39:26] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [20]:
acc*100

79.17944785276073

#HyperTune parameters

In [21]:
booster = ['gbtree','gblinear']
n_estimator = [10,20,30,50,100,200,1000,5000]
max_depth = [2,3,5,10,15,20,50]
learning_rate = [0.01,0.05,0.1,0.2,0.3,0.5,0.7]
min_child_weight = [1,5,10,15,20]
base_score=[0.25,0.5,0.75,1]
aplha = [5,10,15,50,100]
objective = ['reg:squarederror','reg:squaredlogerror']

hyperparameter_grid = {
    'n_estimator' : n_estimator,
    'objective' : objective,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'alpha' : aplha
}


In [22]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6085 entries, 3738 to 7892
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6085 non-null   int64  
 1   CryoSleep     6085 non-null   int64  
 2   Destination   4679 non-null   float64
 3   Age           5966 non-null   float64
 4   VIP           5944 non-null   float64
 5   RoomService   5958 non-null   float64
 6   FoodCourt     5956 non-null   float64
 7   ShoppingMall  5934 non-null   float64
 8   Spa           5957 non-null   float64
 9   VRDeck        5954 non-null   float64
 10  GroupId       6085 non-null   int64  
 11  GrpPassId     6085 non-null   int64  
 12  Deck          4154 non-null   float64
 13  Decknum       5951 non-null   float64
 14  DeckSide      5951 non-null   float64
 15  Expenditure   5443 non-null   float64
 16  Adult         6085 non-null   int64  
dtypes: float64(12), int64(5)
memory usage: 855.7 KB


In [23]:
grid_search_cv = RandomizedSearchCV(
        estimator=model_1,
        param_distributions=hyperparameter_grid,
        cv=5,n_iter=50,
        scoring = 'neg_mean_absolute_error',
        n_jobs=10,
        verbose=5,
        return_train_score=True,
        random_state=2
)
grid_search_cv.fit(x_train,y_train)
grid_search_cv.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   35.8s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:  1.4min
[Parallel(n_jobs=10)]: Done 250 out of 250 | elapsed:  2.3min finished
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:42:32] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




{'objective': 'reg:squarederror',
 'n_estimator': 5000,
 'min_child_weight': 20,
 'max_depth': 15,
 'learning_rate': 0.5,
 'booster': 'gbtree',
 'alpha': 15}

In [24]:
model_2 = xgboost.XGBClassifier(objective = 'reg:squarederror', n_estimator=5000,min_child_weight=20,max_depth=15,learning_rate=0.5,booster='gbtree',alpha=15)
fit_model(model_2,x_train,y_train)
model_2.fit(x_train,y_train)
y_pred = predict_results(model_2,x_test)
acc = calculate_acc(model_2,y_test,y_pred)
acc*100

[14:43:30] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:43:31] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




79.63957055214725

In [53]:
acc

0.7891104294478528

In [25]:
def process(data):
    print(data.shape)
    data[['GroupId','GrpPassId']] = data['PassengerId'].str.split('_',expand=True)
    data[['Deck','Decknum','DeckSide']] = data['Cabin'].str.split('/',expand=True)
    data.loc[(data.Destination == 'TRAPPIST-1e' )  & (data.HomePlanet.isnull()), 'HomePlanet'] ='Earth'
    data.loc[(data.Destination == '55 Cancri e' )  & (data.HomePlanet.isnull()), 'HomePlanet'] = 'Europa'
    data.loc[(data.Destination == 'PSO J318.5-22' ) & (data.HomePlanet.isnull()) , 'HomePlanet'] = 'Earth'
    data['HomePlanet'].fillna(value='Earth',inplace=True)
    #data.dropna(how = 'all',inplace=True)
    data['Expenditure'] = data['RoomService'] + data['FoodCourt'] + data['ShoppingMall'] + data['Spa'] +data['VRDeck']
    child_train_index = data[data['Age'] <= 12].index
    data['Adult'] = 1
    data['Adult'].loc[child_train_index] = 0
    #plt.figure(figsize=(15,8))
    #sns.countplot(x='Destination',hue='Deck',data=data)
    #plt.figure(figsize=(15,8))
    #sns.countplot(x='HomePlanet',hue='Deck',data=data)
    #plt.figure(figsize=(15,8))
    #sns.countplot(x='Destination',hue='Deck',data=data)
    
    
    data.drop(labels=['Name','Cabin'],axis=1,inplace=True)
    cat_cols = list(df.select_dtypes('object').columns.values)
    cat_cols.remove('PassengerId')
    
    tf = {True:1,False:0}
    home_planet = {'Earth':3,'Europa':3,'Mars':1}
    dest_planet = {'TRAPPIST-1e':3,'Cancri e':2,'PSO J318.5-22':1}
    deck = {'F':7,'E':6,'D':5,'C':4,'B':3,'A':2,'T':1}
    port = {'S':0,'P':1}
    
    data['CryoSleep'] = data['CryoSleep'].map(tf)
    data['Destination'] = data['Destination'].map(dest_planet)
    data['HomePlanet'] = data['HomePlanet'].map(home_planet)
    data['VIP'] = data['VIP'].map(tf)
    data['Deck'] = data['Deck'].map(deck)
    data['DeckSide'] = data['DeckSide'].map(port)
    data['Decknum'].fillna(0,inplace=True)
    data['Decknum'] = data['Decknum'].astype(int)
    data['GroupId'].fillna(0,inplace=True)
    data['GroupId'] = data['GroupId'].astype(int)
    data['GrpPassId'].fillna(0,inplace=True)
    data['GrpPassId'] = data['GroupId'].astype(int)
#    data['Transported'] = data['Transported'].map(tf)
    pred = pd.DataFrame(['PassengerId','Transported'])
    #pred['PassengerId'] = df['PassengerId']
    X = data.drop(labels=['PassengerId'],axis=1)
    
    #y = df['Transported']
    passenger_id =df['PassengerId']
    print(passenger_id.shape)
    
    y_pred = model_2.predict(X)
    print(len(y_pred))
    #pred['Transported'] = y_pred
    data_pred = {'PassengerId': data['PassengerId'], 'Transported': y_pred}
    data_pred =pd.DataFrame(data_pred)
    return data_pred
    
    
    
    
    
    
    
    
    

    
    

    

In [26]:
data = pd.read_csv('data/test.csv')
X = process(data)
X.shape

(4277, 13)
(8693,)
4277


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


(4277, 2)

In [27]:
X.head(3)

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1


In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   object
 1   Transported  4277 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 67.0+ KB


In [29]:
#X['GroupId'].value_counts()

"""X['Decknum'].value_counts()
for i in range(len(X)):
    X.loc[i,'Decknum'] = (X.iloc[i,'Decknum'])
    X.loc[i,'GroupId'] = int(X.iloc[i,'GroupId'])"""

"X['Decknum'].value_counts()\nfor i in range(len(X)):\n    X.loc[i,'Decknum'] = (X.iloc[i,'Decknum'])\n    X.loc[i,'GroupId'] = int(X.iloc[i,'GroupId'])"

In [30]:
recon = {0:False,1:True}
X['Transported'] = X['Transported'].map(recon)
X.shape

(4277, 2)

In [149]:
path = 'data/submission.csv'
X.to_csv(path,index=False)

In [150]:
pred.head(3)

,PassengerId,Transported
0,0001_01,1
1,0002_01,0
2,0003_01,0


In [ ]:
y_pred = model_2.predict(X)
    #pred['Transported'] = y_pred
data_pred = {'PassengerId': df['PassengerId'], 'Transported': y_pred}
data_pred =pd.DataFrame(data_pred)

In [31]:
def process_new(data,test,model):
    print(data.shape)
    new_df = data.copy()
    data[['GroupId','GrpPassId']] = data['PassengerId'].str.split('_',expand=True)
    data[['Deck','Decknum','DeckSide']] = data['Cabin'].str.split('/',expand=True)
    data.loc[(data.Destination == 'TRAPPIST-1e' )  & (data.HomePlanet.isnull()), 'HomePlanet'] ='Earth'
    data.loc[(data.Destination == '55 Cancri e' )  & (data.HomePlanet.isnull()), 'HomePlanet'] = 'Europa'
    data.loc[(data.Destination == 'PSO J318.5-22' ) & (data.HomePlanet.isnull()) , 'HomePlanet'] = 'Earth'
    data['HomePlanet'].fillna(value='Earth',inplace=True)
    #data.dropna(how = 'all',inplace=True)
    data['Expenditure'] = data['RoomService'] + data['FoodCourt'] + data['ShoppingMall'] + data['Spa'] +data['VRDeck']
    child_train_index = data[data['Age'] <= 12].index
    data['Adult'] = 1
    data['Adult'].loc[child_train_index] = 0
    #plt.figure(figsize=(15,8))
    #sns.countplot(x='Destination',hue='Deck',data=data)
    #plt.figure(figsize=(15,8))
    #sns.countplot(x='HomePlanet',hue='Deck',data=data)
    #plt.figure(figsize=(15,8))
    #sns.countplot(x='Destination',hue='Deck',data=data)
    
    
    data.drop(labels=['Name','Cabin'],axis=1,inplace=True)
    cat_cols = list(data.select_dtypes('object').columns.values)
    cat_cols.remove('PassengerId')
    
    tf = {True:1,False:0}
    home_planet = {'Earth':3,'Europa':3,'Mars':1}
    dest_planet = {'TRAPPIST-1e':3,'Cancri e':2,'PSO J318.5-22':1}
    deck = {'F':7,'E':6,'D':5,'C':4,'B':3,'A':2,'T':1}
    port = {'S':0,'P':1}
    
    data['CryoSleep'] = data['CryoSleep'].map(tf)
    data['Destination'] = data['Destination'].map(dest_planet)
    data['HomePlanet'] = data['HomePlanet'].map(home_planet)
    data['VIP'] = data['VIP'].map(tf)
    data['Deck'] = data['Deck'].map(deck)
    data['DeckSide'] = data['DeckSide'].map(port)
    data['Decknum'].fillna(0,inplace=True)
    data['Decknum'] = data['Decknum'].astype(int)
    data['GroupId'].fillna(0,inplace=True)
    data['GroupId'] = data['GroupId'].astype(int)
    data['GrpPassId'].fillna(0,inplace=True)
    data['GrpPassId'] = data['GroupId'].astype(int)
#    data['Transported'] = data['Transported'].map(tf)
    pred = pd.DataFrame(['PassengerId','Transported'])
    #pred['PassengerId'] = df['PassengerId']
    data = data.drop(labels=['PassengerId'],axis=1)
    if test == False:
        data.drop(labels=['Transported'],axis=1,inplace=True)
        X = data
        
    else:
        
        

        y_pred = model.predict(data)
        print(len(y_pred))
        #pred['Transported'] = y_pred
        data_pred = {'PassengerId': new_df['PassengerId'], 'Transported': y_pred}
        #print(data_pred.shape)
        data_pred =pd.DataFrame(data_pred)
        print(data_pred.shape)
        X =data_pred
        
    return X

In [33]:
train_df = pd.read_csv('data/train.csv')
processed_data = process_new(train_df,False,False)
processed_data.head(3)

(8693, 14)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,GroupId,GrpPassId,Deck,Decknum,DeckSide,Expenditure,Adult
0,3,0.0,3.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3.0,0,1.0,0.0,1
1,3,0.0,3.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,2,2,7.0,0,0.0,736.0,1
2,3,0.0,3.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,3,3,2.0,0,0.0,10383.0,1


In [34]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   int64  
 1   CryoSleep     8476 non-null   float64
 2   Destination   6711 non-null   float64
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   float64
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  GroupId       8693 non-null   int64  
 11  GrpPassId     8693 non-null   int64  
 12  Deck          5935 non-null   float64
 13  Decknum       8693 non-null   int64  
 14  DeckSide      8494 non-null   float64
 15  Expenditure   7785 non-null   float64
 16  Adult         8693 non-null   int64  
dtypes: float64(12), int64(5)
memory usage: 1.1 MB


In [38]:
X = processed_data
y = train_df['Transported'].map(tf)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
parameters = {'objective' :'reg:squarederror', 'colsample_bytree' : 0.3, 'learning_rate' : 0.1,
                'max_depth' : 5, 'alpha' : 10, 'n_estimator' : 10}
model_1 = train_xgb_model(parameters)
fit_model(model_1,x_train,y_train)
model_1.fit(x_train,y_train)
y_pred = predict_results(model_1,x_test)
acc = calculate_acc(model_1,y_test,y_pred)
acc*100

[14:49:59] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:49:59] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




80.86656441717791

In [39]:
y

0       0
1       1
2       0
3       0
4       1
       ..
8688    0
8689    0
8690    1
8691    0
8692    1
Name: Transported, Length: 8693, dtype: int64

In [41]:
#Retune Model_1
booster = ['gbtree','gblinear']
n_estimator = [10,50,100,200,]
max_depth = [2,3,5,10,15,20,50]
learning_rate = [0.01,0.05,0.1,0.2,0.3,0.5,0.7]
min_child_weight = [1,5,10,15,20]
base_score=[0.25,0.5,0.75,1]
aplha = [5,10,15,50,100]
objective = ['reg:squarederror']

hyperparameter_grid = {
    'n_estimator' : n_estimator,
    'objective' : objective,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'alpha' : aplha
}

grid_search_cv = RandomizedSearchCV(
        estimator=model_1,
        param_distributions=hyperparameter_grid,
        cv=5,n_iter=100,
        scoring = 'neg_mean_absolute_error',
        n_jobs=5,
        verbose=5,
        return_train_score=True,
        random_state=42,
        
)
grid_search_cv.fit(x_train,y_train)
grid_search_cv.best_params_


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   10.5s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:   39.6s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:  1.8min
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed:  3.1min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  4.6min
[Parallel(n_jobs=5)]: Done 638 tasks      | elapsed:  6.5min
[Parallel(n_jobs=5)]: Done 872 tasks      | elapsed:  8.6min
[Parallel(n_jobs=5)]: Done 1142 tasks      | elapsed: 11.7min
[Parallel(n_jobs=5)]: Done 1448 tasks      | elapsed: 15.1min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed: 18.3min
[Parallel(n_jobs=5)]: Done 2168 tasks      | elapsed: 21.7min
[Parallel(n_jobs=5)]: Done 2582 tasks      | elapsed: 25.6min
[Parallel(n_jobs=5)]: Done 3032 tasks      | elapsed: 29.8min
[Parallel(n_jobs=5)]: Done 3518 tasks      | elapsed: 34.2min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed: 39.4min
[P

[15:39:16] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




{'objective': 'reg:squarederror',
 'n_estimator': 200,
 'min_child_weight': 5,
 'max_depth': 15,
 'learning_rate': 0.7,
 'booster': 'gbtree',
 'alpha': 15}

In [45]:
X = processed_data
y = train_df['Transported'].map(tf)
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
parameters = {'objective' :'reg:squarederror', 'colsample_bytree' : 0.3, 'learning_rate' : 0.7,
                'max_depth' : 15, 'alpha' : 15, 'n_estimator' : 200,'min_child_weight': 5,'booster': 'gbtree'}
model_1_tuned = train_xgb_model(parameters)
fit_model(model_1_tuned,x_train,y_train)
model_1_tuned.fit(x_train,y_train)
y_pred = predict_results(model_1_tuned,x_test)
acc = calculate_acc(model_1_tuned,y_test,y_pred)
acc*100

[16:42:44] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:42:45] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




78.87269938650306

In [58]:
test_df = pd.read_csv('data/test.csv')
processed_test_data = process_new(test_df,True)
processed_test_data.map(recon)
processed_test_data.head(3)

(4277, 13)
4277
(4277, 2)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True


In [59]:
#processed_test_data.to_csv('data/submission.csv',index=False)

In [46]:
X = processed_data
y = train_df['Transported'].map(tf)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
parameters = {'objective' :'reg:squarederror', 'colsample_bytree' : 0.3, 'learning_rate' : 0.1,
                'max_depth' : 5, 'alpha' : 10, 'n_estimator' : 10}
model_3 = train_xgb_model(parameters)
fit_model(model_3,x_train,y_train)
model_3.fit(x_train,y_train)
y_pred = predict_results(model_3,x_test)
acc = calculate_acc(model_1,y_test,y_pred)
acc

[16:43:38] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:43:39] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.8052147239263804

In [47]:
#Retune Model_1
booster = ['gbtree','gblinear']
n_estimator = [10,50,100,200,500,1000]
max_depth = [2,3,5,10,15,20,50]
learning_rate = [0.01,0.05,0.1,0.2,0.3,0.5,0.7]
min_child_weight = [1,5,10,15,20]
base_score=[0.25,0.5,0.75,1]
aplha = [5,10,15,50,100]
objective = ['reg:squarederror']

hyperparameter_grid = {
    'n_estimator' : n_estimator,
    'objective' : objective,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'alpha' : aplha
}

grid_search_cv = RandomizedSearchCV(
        estimator=model_3,
        param_distributions=hyperparameter_grid,
        cv=5,n_iter=100,
        scoring = 'neg_mean_absolute_error',
        n_jobs=5,
        verbose=5,
        return_train_score=True,
        random_state=42,
        
)
grid_search_cv.fit(x_train,y_train)
grid_search_cv.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   10.0s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:   37.7s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:  1.4min
[Parallel(n_jobs=5)]: Done 278 tasks      | elapsed:  2.5min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  4.1min
[Parallel(n_jobs=5)]: Done 500 out of 500 | elapsed:  4.7min finished
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:03] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




{'objective': 'reg:squarederror',
 'n_estimator': 100,
 'min_child_weight': 5,
 'max_depth': 15,
 'learning_rate': 0.5,
 'booster': 'gbtree',
 'alpha': 50}

In [67]:
X = processed_data
y = train_df['Transported'].map(tf)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
"""parameters = {'objective' :'reg:squarederror', 'colsample_bytree' : 0.3, 'learning_rate' : 0.5,
                'max_depth' : 40, 'alpha' : 10, 'n_estimator' : 100}"""
parameters = {'objective' :'reg:squarederror', 'colsample_bytree' : 0.3, 'learning_rate' : 0.5,
                'max_depth' : 45, 'alpha' : 10, 'n_estimator' : 200}
model_3_tuned = train_xgb_model(parameters)
fit_model(model_3_tuned,x_train,y_train)
model_3_tuned.fit(x_train,y_train)
y_pred = predict_results(model_3_tuned,x_test)
acc = calculate_acc(model_3_tuned,y_test,y_pred)
acc*100

[16:57:44] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:57:45] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimator", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




81.17331288343557

In [69]:
test_df = pd.read_csv('data/test.csv')
processed_test_data = process_new(test_df,True,model_3_tuned)
processed_test_data['Transported'] = processed_test_data['Transported'].map(recon)
processed_test_data.to_csv('data/submission_mode_3_tuned.csv',index=False)
processed_test_data.head(3)


(4277, 13)
4277
(4277, 2)


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True


In [85]:
#filename = 'model_artifacts/mod_81_acc.sav'
#pickle.dump(model_3, open(filename, 'wb'))